In [22]:
import requests
from openpyxl import Workbook
from random import choice
from bs4 import BeautifulSoup
from string import ascii_lowercase
import re

wb = Workbook()
ws1 = wb.create_sheet('Giełda')
ws2 = wb.create_sheet('Linki')
ws3 = wb.create_sheet('Filmweb')

In [28]:
def generujString(n):
    return "".join(choice(ascii_lowercase) for i in range(n))

ws1.cell(row=1, column=1).value = 'Kod'
ws1.cell(row=1, column=2).value = 'Kurs'
ws1.cell(row=1, column=3).value = 'Zmiana procentowa'
ws1.cell(row=1, column=4).value = 'Transakcje'

kody = [generujString(3) for _ in range(5)]
s = requests.Session()
znalezione_kody = []
for kod in kody:
    print(kod)
    parametry = {"s": kod}
    odpowiedz = requests.get("http://stooq.pl/q/", params=parametry)
    beautiful_soup = BeautifulSoup(odpowiedz.content, 'html.parser')
    if len(beautiful_soup.body.findAll('span', id='c2')) != 0:
        link_content = beautiful_soup.find('font',id='f16').find('a',href=True)
        if link_content.text == 'symbol waloru':
            print("Nie odnaleziono kodu")
        else:
            poprawny_kod = link_content['href'][5:]
            znalezione_kody.append(poprawny_kod)
            print(f'Znaleziono kod podobny do: {poprawny_kod}')
    else:
        znalezione_kody.append(poprawny_kod)
row = 2
for kod in znalezione_kody:
    print(kod)
    parametry = {"s": kod}
    odpowiedz = requests.get("http://stooq.pl/q/", params=parametry)
    beautiful_soup = BeautifulSoup(odpowiedz.content, 'html.parser')
    print(f"{price}, {percent_change}, {transactions}")
    ws1.cell(row=row, column=1).value = kod
    ws1.cell(row=row, column=2).value = beautiful_soup.find(text="Kurs").parent.find("span").text 
    ws1.cell(row=row, column=3).value =  beautiful_soup.find(text="Zmiana").parent.findAll("span")[1].text 
    ws1.cell(row=row, column=4).value =  beautiful_soup.find(text="Transakcje").parent.find("span").text
    row += 1

skm
Znaleziono kod podobny do: skm.us
zaw
Nie odnaleziono kodu
scr
Znaleziono kod podobny do: scrf.uk
wgn
Nie odnaleziono kodu
ujq
Nie odnaleziono kodu
skm.us
18.8800, (-2.83%), 46
scrf.uk
18.8800, (-2.83%), 46


In [51]:
url = "http://www.wme.amw.gdynia.pl/"
request = requests.get(url)
linki = set()
beautiful_soup = BeautifulSoup(request.content, 'html.parser')
for a in beautiful_soup.find_all("a"):
    link = a.get('href')
    linki.add(link)
    print(link)
row = 1
for link in linki:
    ws2.cell(row,column=1).value = link
    row +=1

None
#ja-content
#ja-mainnav
#ja-col1
#ja-col2
/index.php
http://www.facebook.com/wme.amw
https://twitter.com/wme_amw
http://matlab.amw.ont.com.pl/
/ieiao/baza-dydaktyczna/493-laboratorium-ipms-system-zarzdzania-zintegrowan-platform-okrtu-
/ieiao/baza-dydaktyczna/494-laboratorium-sterownikow-plc
http://www.wme.amw.gdynia.pl/ieiao/baza-dydaktyczna/18-lr
/index.php?view=article&catid=3:aktualnosci&id=523:rekrutacja-uzupeniajca-do-suby-kandydackiej&tmpl=component&print=1&layout=default&page=
/index.php?view=article&catid=3:aktualnosci&id=523:rekrutacja-uzupeniajca-do-suby-kandydackiej&format=pdf
/aktualnosci/523-rekrutacja-uzupeniajca-do-suby-kandydackiej
/index.php?view=article&catid=3:aktualnosci&id=505:ii-stopie-studiow-na-kierunku-automatyka-i-robotyka&tmpl=component&print=1&layout=default&page=
/index.php?view=article&catid=3:aktualnosci&id=505:ii-stopie-studiow-na-kierunku-automatyka-i-robotyka&format=pdf
/index.php?view=article&catid=3:aktualnosci&id=502:przedmioty-ogolnouczelniane

In [36]:
url = "https://www.filmweb.pl/film/Parasite-2019-798143"
request = requests.get(url)
beautiful_soup = BeautifulSoup(request.content, 'html.parser')

filmweb = {}
filmweb['ocena'] = beautiful_soup.find("span", itemprop="ratingValue").text
filmweb['reżyser'] = beautiful_soup.find(text="reżyseria:").next_element.find("a").text
filmweb['premiear'] = beautiful_soup.find("h1", class_="filmTitle").find('span').text[1:5]
boxoffice = beautiful_soup.find(text="boxoffice:").next_element.text
filmweb['box office']  = re.sub(r'(top #\d+)', '', boxoffice)

row = 1
for klucz, wartosc in filmweb.items():
    ws3.cell(row,column=1).value = klucz
    ws3.cell(row,column=2).value = wartosc
    row += 1

In [ ]:
wb.save('Lab4.xlsx')